In [423]:
#Read Input & Assign Variables
import time
import csv
import math
with open('speed_to_data_map.csv', newline='') as mapping:
    speed_map = tuple(csv.reader(mapping))
    
speed_data_map = []
for i in range (0,len(speed_map)):
    speed_data_map = (*speed_data_map, int(speed_map[i][1]))


In [424]:
def size_index(data,K):
    index_map = []
    for i in range (0,K):
        max_index = data.index(max(data))
        index_map.append(max_index)
        data[max_index] = -1
    return index_map

In [425]:
def target_factor(M,init_speed,factor,total_data,index_map,users,flag,avg_speed,instance,data_remaining):
    sent_data = 0
    collocated_factor_list = [0]*M
    for i in range (0,users):
        collocated_factor_list[i] = factor[index_map[i]]/100
    
    for i in range (0,users):
        collocated_factor = collocated_factor_list[i] * sum((collocated_factor_list[:i]+collocated_factor_list[i+1:]))
        speed = init_speed[index_map[i]] * (1-collocated_factor)
        speed_floor = math.floor(speed + 10e-6)
        sent_data = sent_data + speed_data_map[speed_floor]
        if flag == 1:
            data_remaining[index_map[i]] = data_remaining[index_map[i]] - speed_data_map[speed_floor]
            if (data_remaining[index_map[i]] < 0):
                data_remaining[index_map[i]] = 0
            avg_speed[index_map[i]] = ((avg_speed[index_map[i]])*instance[index_map[i]] + speed)/(instance[index_map[i]]+1)
            instance[index_map[i]] = instance[index_map[i]] + 1

    target_curr = (math.floor((sent_data/total_data)*100))
    return target_curr


In [426]:
def write(M,N,grid,avg_speed,data_loss,total_data,alpha,start_time):
    print(grid)
    execution_time = time.time() - start_time
    print(execution_time)
    with open('example.csv', 'w') as file:
        writer = csv.writer(file)
    
   # writer.writerows(grid)
    
    # for i in range (0,M):
    #     write_data = []
    #     for j in range (0,N):
            
            

In [427]:
def main():
    start_time = time.time()
    with open('toy_testcase.csv', newline='') as input:
        data = tuple(csv.reader(input))

    M = int(data[0][0])
    N = int(data[0][1])
    K = int(data[1][0])
    alpha = int(data[2][0])

    init_speed = ()
    data_size = ()
    factor = ()

    for i in range (3,3+K):
        init_speed = (*init_speed, int(data[i][1]))
        data_size = (*data_size, int(data[i][2]))
        factor = (*factor, int(data[i][3]))
    total_data = sum(list(data_size))
    data_remaining = list(data_size)

    grid = [[0]*M]*N
    avg_speed = [0]*K
    instance = [0]*K
    data_loss = [0]*K
    
    target_ideal = 100/N
    for i in range (0,N):       #Columns 
        user_index_map = [0]*M
        index_map_temp = [0]*M
        data_temp = data_remaining.copy()
        size_index_map = size_index(data_temp,K)         #shows the index of largest and the subsequent largest data_size 
        margin = 100
        for j in range (0,M):
            index_map_temp[j] = size_index_map[j]  
            target_curr = target_factor(M,init_speed,factor,total_data,index_map_temp,j+1,0,avg_speed,instance,data_remaining)
            if abs(target_curr-target_ideal) > margin:
                target_factor(M,init_speed,factor,total_data,user_index_map,j,1,avg_speed,instance,data_remaining)
                grid[i] = user_index_map.copy()
                break
            else:
                margin = abs(target_curr - target_ideal)
                user_index_map = index_map_temp.copy()
        
    data_loss = data_remaining.copy()
    write(M,N,grid,avg_speed,data_loss,total_data,alpha,start_time)

In [428]:
if __name__ == '__main__':
    main()
    


[[3, 2, 0], [3, 1, 0], [3, 1, 0], [3, 0, 0], [2, 1, 0]]
0.0009958744049072266
